In [15]:
import pandas as pd
import numpy as np
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error,r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
import argparse
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time
from sklearn.pipeline import  Pipeline, make_pipeline

from transformation import RemoveSkewnessKurtosis, Standardize, ImputeNumeric


from sklearn.utils.validation import column_or_1d

# Import Data
path = '/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality.csv'
data = pd.read_csv(path).iloc[:,1:]
X = data.drop(columns = ['CVD_readmission', 'readmission within 300 days'])
y = column_or_1d(data[['readmission within 300 days']])
 


# Split Train and Test
transform_steps = [("ImputeNumeric", ImputeNumeric()),
                   ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis()),
                   ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

X = transform_pipeline.transform(X)

KeyError: "['CVD_readmission', 'readmission within 300 days'] not found in axis"

In [2]:
# Grid Search CV
import numpy as np
import pandas as pd
import scipy as sp
import copy,os,sys,psutil
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import dump_svmlight_file
from sklearn.linear_model import LogisticRegression
 
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from grid_search_cv import *

gsearch = LGBM_Grid_CV(X,y, LGBM_param)

In [12]:
d2=pd.DataFrame(X['Internalpatientid'])

In [13]:
d2['new'] =[0]*84536

In [16]:
data

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,AO,CVD,unique_admitting_specialty,unique_discharging_specialty,...,stay_mean,stay_std,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,died_by_cvd,Age at death
0,3,2,407.50,2,0,1,0,1,2,2,...,203.750000,285.558003,0.67,0,1,0,1,0,0,82.284691
1,4,1,7.15,1,0,1,0,1,1,1,...,7.150000,0.000000,1.00,0,0,0,1,0,0,84.190359
2,5,2,8.45,2,0,1,0,1,2,1,...,4.225000,4.504270,2.00,2,0,0,0,0,0,76.506124
3,10,1,0.33,1,0,0,1,0,1,1,...,0.330000,0.000000,1.00,0,1,0,0,0,0,72.384665
4,15,4,127.10,3,0,3,1,1,4,3,...,31.775000,55.299774,0.21,2,0,1,1,0,0,86.598848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61003,169053,12,135.47,3,0,3,0,1,6,6,...,11.289167,17.461613,2.40,4,0,0,7,0,0,81.349588
61004,169055,1,6.47,1,0,1,0,1,1,1,...,6.470000,0.000000,1.00,0,1,0,0,0,0,68.572433
61005,169060,8,45.38,2,0,6,0,1,5,2,...,5.672500,7.616855,0.53,5,1,0,1,1,0,71.832486
61006,169062,12,178.63,4,1,0,1,0,7,5,...,14.885833,16.735598,3.00,10,2,0,0,0,0,74.181399


In [3]:
print(gsearch.best_params_)

{'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_samples': 10, 'n_estimators': 100, 'subsample': 0.7}
